In [1]:
import pandas as pd
import datetime as dt
import re, math
import numpy as np

In [2]:
df = pd.read_csv('Stack_Overflow_Jobs.csv')
df.head()

,company,location,perks,posted,request_date,salary,tags,title
0,Kindred AI,"\r\r\n - \r\r\nToronto, ON, Canada...",NaN,< 1h ago,2018-11-15 13:42:44.056522,C$90k - 110k,"scrum, agile",Technical Program Manager
1,Articulate Inc.,\r\r\n - \r\r\nNo office location ...,Remote,< 1h ago,2018-11-15 13:42:44.058503,NaN,"node.js, amazon-web-services, terraform, docke...",Platform Engineer
2,Challengermode,"\r\r\n - \r\r\nStockholm, Sweden ...",Visa sponsor,< 1h ago,2018-11-15 13:42:44.059503,SEK 420k - 624k\r\r\n\r\r\n ...,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup
3,Compliance Solutions Strategies,"\r\r\n - \r\r\nNew York, NY",NaN,< 1h ago,2018-11-15 13:42:44.061502,NaN,"c#, .net, javascript, node.js",Front to back Developer
4,Celonis SE,"\r\r\n - \r\r\nMünchen, Germany ...",NaN,< 1h ago,2018-11-15 13:42:44.063500,NaN,"java, spring, java-ee, rest, springboot",Senior Java Developer


In [3]:
df.shape

(1050, 8)

### Processando a coluna Location

In [4]:
def clear_location(x):
    if 'No office location' in x:
        return None
    
    loc = x.split()
    if '-' in loc:
        loc.remove('-')
    return ' '.join(loc)

df['location'] = df['location'].apply(clear_location)
df.head()

,company,location,perks,posted,request_date,salary,tags,title
0,Kindred AI,"Toronto, ON, Canada",NaN,< 1h ago,2018-11-15 13:42:44.056522,C$90k - 110k,"scrum, agile",Technical Program Manager
1,Articulate Inc.,None,Remote,< 1h ago,2018-11-15 13:42:44.058503,NaN,"node.js, amazon-web-services, terraform, docke...",Platform Engineer
2,Challengermode,"Stockholm, Sweden",Visa sponsor,< 1h ago,2018-11-15 13:42:44.059503,SEK 420k - 624k\r\r\n\r\r\n ...,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup
3,Compliance Solutions Strategies,"New York, NY",NaN,< 1h ago,2018-11-15 13:42:44.061502,NaN,"c#, .net, javascript, node.js",Front to back Developer
4,Celonis SE,"München, Germany",NaN,< 1h ago,2018-11-15 13:42:44.063500,NaN,"java, spring, java-ee, rest, springboot",Senior Java Developer


### Criando a coluna Country

In [5]:
def separate_country(row):
    if not row.location:
        row['location'], row['country'] = (None, None)
    else:
        row['location'], _, row['country'] = row.location.partition(', ')
        
    return row
    
df = df.apply(separate_country, axis=1)
df.head()

,company,location,perks,posted,request_date,salary,tags,title,country
0,Kindred AI,Toronto,NaN,< 1h ago,2018-11-15 13:42:44.056522,C$90k - 110k,"scrum, agile",Technical Program Manager,"ON, Canada"
1,Articulate Inc.,None,Remote,< 1h ago,2018-11-15 13:42:44.058503,NaN,"node.js, amazon-web-services, terraform, docke...",Platform Engineer,None
2,Challengermode,Stockholm,Visa sponsor,< 1h ago,2018-11-15 13:42:44.059503,SEK 420k - 624k\r\r\n\r\r\n ...,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup,Sweden
3,Compliance Solutions Strategies,New York,NaN,< 1h ago,2018-11-15 13:42:44.061502,NaN,"c#, .net, javascript, node.js",Front to back Developer,NY
4,Celonis SE,München,NaN,< 1h ago,2018-11-15 13:42:44.063500,NaN,"java, spring, java-ee, rest, springboot",Senior Java Developer,Germany


### Processando a coluna Posted

In [6]:
df.posted.unique()

array(['< 1h ago', '1h ago', '2h ago', '3h ago', '5h ago', '4h ago',
       '6h ago', '7h ago', '8h ago', '10h ago', '9h ago', '11h ago',
       '12h ago', '13h ago', '14h ago', '15h ago', '16h ago', '18h ago',
       '17h ago', '19h ago', '20h ago', '22h ago', '21h ago', '23h ago',
       'yesterday', '2d ago', '3d ago'], dtype=object)

In [7]:
def replace_times_ago_to_date(row):
    times_ago, req_date = row['posted'], row['request_date']
    
    req_date = dt.datetime.strptime(req_date, "%Y-%m-%d %H:%M:%S.%f")
    numero = re.sub('[^\d]', '',times_ago)

    if 'h ago' in times_ago:
        return req_date - dt.timedelta(hours= int(numero))
    elif 'd ago' in times_ago:
        return req_date - dt.timedelta(days= int(numero))
    else:
        return req_date - dt.timedelta(days= 1)
                 
df['posted'] = df.apply(replace_times_ago_to_date, axis=1)
df.head()

,company,location,perks,posted,request_date,salary,tags,title,country
0,Kindred AI,Toronto,NaN,2018-11-15 12:42:44.056522,2018-11-15 13:42:44.056522,C$90k - 110k,"scrum, agile",Technical Program Manager,"ON, Canada"
1,Articulate Inc.,None,Remote,2018-11-15 12:42:44.058503,2018-11-15 13:42:44.058503,NaN,"node.js, amazon-web-services, terraform, docke...",Platform Engineer,None
2,Challengermode,Stockholm,Visa sponsor,2018-11-15 12:42:44.059503,2018-11-15 13:42:44.059503,SEK 420k - 624k\r\r\n\r\r\n ...,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup,Sweden
3,Compliance Solutions Strategies,New York,NaN,2018-11-15 12:42:44.061502,2018-11-15 13:42:44.061502,NaN,"c#, .net, javascript, node.js",Front to back Developer,NY
4,Celonis SE,München,NaN,2018-11-15 12:42:44.063500,2018-11-15 13:42:44.063500,NaN,"java, spring, java-ee, rest, springboot",Senior Java Developer,Germany


In [8]:
df = df.drop(['request_date'], axis=1)

### Processando a coluna Salary

In [9]:
# Valor para conversão de uma moeda para Real
moedas = {
    'R$': 1, # Real
    'C$': 2.88, # Dolar Canadense
    '£': 4.91, # Libra esterlina, Reino Unido
    '€': 4.30, # Euro
    '₹': 0.052, # Rúpia indiana
    'A$': 2.75, # Dolar Australiano
    'zł': 1, # Zloti polonês
    'SEK': 0.42, # Coroa Sueca
    'CHF': 3.76, # Franco suíço
    'MXN': 0.19, # Peso mexicano
    'R': 0.26, # Rand Sul-Africano
    '$': 3.78, # Dolar Americano
}

In [10]:
def clear_salary(row):
    row['moeda_original'] = '-'
    
    if isinstance(row['salary'], float):
        return row
    
    for moeda in moedas.keys():
        if moeda in row['salary']:
            min_max_list = re.sub('[^\d-]','', row['salary']).split('-')

            if len(min_max_list) == 2:
                row['salary_min'], row['salary_max'] = tuple(min_max_list)
            else:
                row['salary_max'] = min_max_list[0]

            row['moeda_original'] = moeda
            break
    
    return row

df = df.apply(clear_salary, axis=1)
df.head()

,company,country,location,moeda_original,perks,posted,salary,salary_max,salary_min,tags,title
0,Kindred AI,"ON, Canada",Toronto,C$,NaN,2018-11-15 12:42:44.056522,C$90k - 110k,110,90,"scrum, agile",Technical Program Manager
1,Articulate Inc.,None,None,-,Remote,2018-11-15 12:42:44.058503,NaN,NaN,NaN,"node.js, amazon-web-services, terraform, docke...",Platform Engineer
2,Challengermode,Sweden,Stockholm,SEK,Visa sponsor,2018-11-15 12:42:44.059503,SEK 420k - 624k\r\r\n\r\r\n ...,624,420,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup
3,Compliance Solutions Strategies,NY,New York,-,NaN,2018-11-15 12:42:44.061502,NaN,NaN,NaN,"c#, .net, javascript, node.js",Front to back Developer
4,Celonis SE,Germany,München,-,NaN,2018-11-15 12:42:44.063500,NaN,NaN,NaN,"java, spring, java-ee, rest, springboot",Senior Java Developer


### Replace Countries

In [11]:
df['country'] = df['country'].replace({
    'Deutschland': 'Alemanha',
    'Germany': 'Alemanha',
    'UK': 'Reino Unido',
    'Netherlands': 'Holanda',
    'Sweden': 'Suécia',
    'France': 'França',
    'Schweiz': 'Suíça',
    'Norway': 'Noruega',
    'Switzerland': 'Suíça',
    'Poland': 'Polônia',
    'Romania': 'Romênia',
    'Belgium': 'Bélgica',
    'Österreich': 'Áustria'
})

### Replace Contry EUA

In [12]:
df['country'] = df['country'].replace({
    'TX': 'Texas, EUA',
    'CA': 'Califórnia, EUA',
    'VA': 'Virgínia, EUA',
    'NY': 'Nova York, EUA',
    'CO': 'Colorado, EUA',
    'PA': 'Pensilvânia, EUA',
    'MA': 'Massachusetts, EUA',
    'IL': 'Illinois, EUA',
    'NJ': 'Nova Jersey, EUA',
    'UT': 'Utah, EUA',
    'OH': 'Ohio, EUA',
    'FL': 'Flórida, EUA',
    'WI': 'Wisconsin, EUA',
    'DC': 'Washington, EUA',
    'TN': 'Tennessee, EUA',
    'MI': 'Michigan, EUA',
    'WA': 'Washington, EUA',
    'MD': 'Maryland, EUA',
    'IN': 'Indiana, EUA',
    'AZ': 'Arizona, EUA',
    'DE': 'Delaware, EUA',
    'GA': 'Georgia, EUA',
    'MO': 'Missouri, EUA',
    'NC': 'Carolina do Norte, EUA'
})

### Convertendo a moeda para Real
   - Converte cada valor para a moeda Real de acordo com a cotação definida em _moedas_
   - Coverte o salário anual para mensal
   - Calcula a média entre o mínimo e máximo

In [13]:
df[['salary_min', 'salary_max', 'salary']] = df[['salary_min', 'salary_max', 'salary']].fillna(0)
df.head()

,company,country,location,moeda_original,perks,posted,salary,salary_max,salary_min,tags,title
0,Kindred AI,"ON, Canada",Toronto,C$,NaN,2018-11-15 12:42:44.056522,C$90k - 110k,110,90,"scrum, agile",Technical Program Manager
1,Articulate Inc.,None,None,-,Remote,2018-11-15 12:42:44.058503,0,0,0,"node.js, amazon-web-services, terraform, docke...",Platform Engineer
2,Challengermode,Suécia,Stockholm,SEK,Visa sponsor,2018-11-15 12:42:44.059503,SEK 420k - 624k\r\r\n\r\r\n ...,624,420,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup
3,Compliance Solutions Strategies,"Nova York, EUA",New York,-,NaN,2018-11-15 12:42:44.061502,0,0,0,"c#, .net, javascript, node.js",Front to back Developer
4,Celonis SE,Alemanha,München,-,NaN,2018-11-15 12:42:44.063500,0,0,0,"java, spring, java-ee, rest, springboot",Senior Java Developer


In [14]:
def converter_para_real(row):
    moeda = 0
    if row['moeda_original'] != '-':
        moeda = moedas[row['moeda_original']]
        
    row['salary_min'] = math.floor((moeda * int(row['salary_min'])) / 12)
    row['salary_max'] = math.ceil((moeda * int(row['salary_max'])) / 12)
    row['salary_mean'] = row[['salary_min','salary_max']].mean()
    return row

df = df.apply(converter_para_real, axis=1)
df = df.drop(['salary'], axis=1)
df.head()

,company,country,location,moeda_original,perks,posted,salary_max,salary_min,tags,title,salary_mean
0,Kindred AI,"ON, Canada",Toronto,C$,NaN,2018-11-15 12:42:44.056522,27,21,"scrum, agile",Technical Program Manager,24.0
1,Articulate Inc.,None,None,-,Remote,2018-11-15 12:42:44.058503,0,0,"node.js, amazon-web-services, terraform, docke...",Platform Engineer,0.0
2,Challengermode,Suécia,Stockholm,SEK,Visa sponsor,2018-11-15 12:42:44.059503,22,14,"c#, asp.net, angularjs, javascript, .net",Fullstack Web Developer at Esports Tech Startup,18.0
3,Compliance Solutions Strategies,"Nova York, EUA",New York,-,NaN,2018-11-15 12:42:44.061502,0,0,"c#, .net, javascript, node.js",Front to back Developer,0.0
4,Celonis SE,Alemanha,München,-,NaN,2018-11-15 12:42:44.063500,0,0,"java, spring, java-ee, rest, springboot",Senior Java Developer,0.0


In [15]:
df = df[['title','tags', 'perks', 'country','location', 'company', 'posted', 'moeda_original', 'salary_min', 'salary_max', 'salary_mean']]

In [16]:
df.to_csv('Stack_Overflow_Jobs2.csv', index=False)

### Criando Data Frame para as Tags

In [17]:
job_tags = []

def separate_tags(row):
    tags_dic = {'id': row.name}

    if not isinstance(row['tags'], float):
        tags = row['tags']
        
        for tag in tags.split(', '):
            tags_dic[tag] = 1
    
    job_tags.append(tags_dic)
    return row
    
    
_ = df.apply(separate_tags, axis=1)

In [18]:
del job_tags[0]

In [19]:
df_tags = pd.DataFrame(job_tags)

In [20]:
df_tags.shape

(1050, 607)

In [21]:
df_tags.head()

,.net,.net-4.0,.net-core,3-tier,3d,4,ab-testing,abap,accessibility,acoustics,...,workday,workfusion,wpf,wxwidgets,xamarin,xcode,xml,yocto,yui,zend-framework
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_tags.to_csv('jobs_tags.csv', index=False)